In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re

# Input data files 

Using TensorFlow backend.


In [2]:
data=pd.read_csv(r'C:\Users\Varun Kurup\Documents\testdatamodoldrnn.csv')

In [3]:
data.head()

,Text,Label
0,The booking is confirmed,2
1,booking is not confirmed,0
2,booking is rejected,0
3,booking is accepted,2
4,accepted,2


In [4]:

data.iloc[:,0] = data.iloc[:,0].apply(lambda x: x.lower())
# removing special chars
data.iloc[:,0] = data.iloc[:,0].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
#
data.head()

,Text,Label
0,the booking is confirmed,2
1,booking is not confirmed,0
2,booking is rejected,0
3,booking is accepted,2
4,accepted,2


C:\Users\Varun Kurup\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


0
0
0


,Text,Label
0,the booking is confirmed,2
1,booking is not confirmed,0
2,booking is rejected,0
3,booking is accepted,2
4,accepted,2


In [5]:
max_fatures = 200
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data.iloc[:,0].values)
X = tokenizer.texts_to_sequences(data.iloc[:,0].values)
X = pad_sequences(X)
X[:2]

array([[0, 0, 0, 0, 0, 0, 0, 4, 1, 2, 6],
       [0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 6]])

In [6]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 11, 128)           25600     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 11, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 591       
Total params: 280,991
Trainable params: 280,991
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
Y = pd.get_dummies(data.iloc[:,-1]).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.30, random_state = 84)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1123, 11) (1123, 3)
(482, 11) (482, 3)


In [8]:
batch_size = 128
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
1123/1123 [==============================] - 3s 3ms/step - loss: 0.9900 - acc: 0.5841
Epoch 2/15
1123/1123 [==============================] - 1s 512us/step - loss: 0.8353 - acc: 0.5788
Epoch 3/15
1123/1123 [==============================] - 1s 517us/step - loss: 0.7815 - acc: 0.6830
Epoch 4/15
1123/1123 [==============================] - 1s 496us/step - loss: 0.6644 - acc: 0.7391
Epoch 5/15
1123/1123 [==============================] - 1s 524us/step - loss: 0.5211 - acc: 0.8299
Epoch 6/15
1123/1123 [==============================] - 1s 536us/step - loss: 0.3908 - acc: 0.8664
Epoch 7/15
1123/1123 [==============================] - 1s 519us/step - loss: 0.2586 - acc: 0.9261
Epoch 8/15
1123/1123 [==============================] - 1s 545us/step - loss: 0.1947 - acc: 0.9564
Epoch 9/15
1123/1123 [==============================] - 1s 540us/step - loss: 0.1773 - acc: 0.9564
Epoch 10/15
1123/1123 [==============================] - 1s 519

In [9]:
X.shape

(1605, 11)

In [10]:
Y_pred = model.predict_classes(X_test,batch_size = batch_size)

In [11]:
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix [[214   0   6]
 [  1  28   7]
 [  4   1 221]]
              precision    recall  f1-score   support

           0       0.98      0.97      0.97       220
           1       0.97      0.78      0.86        36
           2       0.94      0.98      0.96       226

   micro avg       0.96      0.96      0.96       482
   macro avg       0.96      0.91      0.93       482
weighted avg       0.96      0.96      0.96       482



Epoch 1/15
1123/1123 [==============================] - 1s 590us/step - loss: 0.0407 - acc: 0.9920
Epoch 2/15
1123/1123 [==============================] - 1s 541us/step - loss: 0.0399 - acc: 0.9920
Epoch 3/15
1123/1123 [==============================] - 1s 506us/step - loss: 0.0399 - acc: 0.9929
Epoch 4/15
1123/1123 [==============================] - 1s 511us/step - loss: 0.0379 - acc: 0.9929
Epoch 5/15
1123/1123 [==============================] - 1s 521us/step - loss: 0.0387 - acc: 0.9929
Epoch 6/15
1123/1123 [==============================] - 1s 491us/step - loss: 0.0346 - acc: 0.9938
Epoch 7/15
1123/1123 [==============================] - 1s 494us/step - loss: 0.0315 - acc: 0.9947
Epoch 8/15
1123/1123 [==============================] - 1s 503us/step - loss: 0.0332 - acc: 0.9938
Epoch 9/15
1123/1123 [==============================] - 1s 504us/step - loss: 0.0302 - acc: 0.9938
Epoch 10/15
1123/1123 [==============================] - 1s 492us/step - loss: 0.0312 - acc: 0.9920
Epoch 11/

TypeError: Cannot clone object '<keras.engine.sequential.Sequential object at 0x000002068AD6BBE0>' (type <class 'keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.